In [13]:
pip install cassandra-driver

Note: you may need to restart the kernel to use updated packages.


# Importando Bibliotecas

In [14]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType
from pyspark.sql.window import Window
import pandas as pd

In [15]:
from cassandra.cluster import Cluster
from cassandra.connection import EndPoint
cluster = Cluster(['34.95.160.101'])
                               

session = cluster.connect('irregularidades')

query1 = "SELECT * FROM irreg_2014_2021"

saida = session.execute(query1)
dados = []
for i in saida:
    dados.append(i)
 
df_irreg = pd.DataFrame(dados)
cluster.shutdown()
df_irreg.drop(columns=['chave'],inplace=True)
df_irreg

,ano,categoria,instituicao_financeira,irregularidade,periodo,qtd_reclamacoes_nao_reguladas,qtd_reclamacoes_reguladas_outras,qtd_reclamacoes_reguladas_procedentes,qtd_total_reclamacoes,tipo
0,2015,Menos de dois milhões de clientes,BANCO DAYCOVAL S A,Insatisfação com o atendimento prestado pelo SAC,1º semestre,33,0,0,33,Banco/financeira
1,2020,Menos de quatro milhões de clientes,SAFRA (conglomerado),Irregularidades relacionadas ao fornecimento d...,2º trimestre,0,45,21,66,Conglomerado
2,2019,Mais de quatro milhões de clientes,SANTANDER (conglomerado),Abertura de conta de depósito com documentação...,2º trimestre,0,3,16,19,Conglomerado
3,2019,Mais de quatro milhões de clientes,BMG (conglomerado),Bloqueio de margem consignável sem utilização ...,4º trimestre,36,0,0,36,Conglomerado
4,2019,Menos de quatro milhões de clientes,DACASA FINANCEIRA (conglomerado),Recusa de produtos ou serviços,1º trimestre,1,0,0,1,Conglomerado
...,...,...,...,...,...,...,...,...,...,...
52113,2018,Mais de quatro milhões de clientes,VOTORANTIM (conglomerado),Cobranças diversas em operação de crédito cons...,4º trimestre,19,0,0,19,Conglomerado
52114,2021,Demais bancos e financeiras,BANCO SICOOB (conglomerado),Oferta ou prestação de informação sobre operaç...,3º trimestre,0,1,0,1,Conglomerado
52115,2015,Menos de dois milhões de clientes,BRB (conglomerado),Descumprimento de prazo de resposta da reclama...,1º semestre,0,6,0,6,Conglomerado
52116,2019,Menos de quatro milhões de clientes,BANESTES (conglomerado),Recusa ou dificuldade de acesso aos canais de ...,3º trimestre,0,1,1,2,Conglomerado


# Configurando a Spark Session

In [16]:
spark = (SparkSession
        .builder
        .master('local')
        .appName('Irregularidades_Pyspark')
        .config('spark.ui.port', '4050')
        .getOrCreate()
        )

In [17]:
df = spark.createDataFrame(df_irreg)
df.show(5,truncate=False)

+----+-----------------------------------+--------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------+--------------------------------+-------------------------------------+---------------------+----------------+
|ano |categoria                          |instituicao_financeira          |irregularidade                                                                                                                                        |periodo     |qtd_reclamacoes_nao_reguladas|qtd_reclamacoes_reguladas_outras|qtd_reclamacoes_reguladas_procedentes|qtd_total_reclamacoes|tipo            |
+----+-----------------------------------+--------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-

22/01/23 20:39:04 WARN org.apache.spark.scheduler.TaskSetManager: Stage 13 contains a task of very large size (9891 KiB). The maximum recommended task size is 1000 KiB.


In [18]:
df.printSchema()

root
 |-- ano: long (nullable = true)
 |-- categoria: string (nullable = true)
 |-- instituicao_financeira: string (nullable = true)
 |-- irregularidade: string (nullable = true)
 |-- periodo: string (nullable = true)
 |-- qtd_reclamacoes_nao_reguladas: long (nullable = true)
 |-- qtd_reclamacoes_reguladas_outras: long (nullable = true)
 |-- qtd_reclamacoes_reguladas_procedentes: long (nullable = true)
 |-- qtd_total_reclamacoes: long (nullable = true)
 |-- tipo: string (nullable = true)



In [19]:
#Vizualização do número de vezes que cada categoria de irregularidades aparece,
# ordenada de forma decrescente
df.groupBy('categoria').count().sort("count", ascending=False).show()

22/01/23 20:39:04 WARN org.apache.spark.scheduler.TaskSetManager: Stage 14 contains a task of very large size (9891 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----+
|           categoria|count|
+--------------------+-----+
|Menos de quatro m...|20088|
|Mais de quatro mi...|13209|
|Menos de dois mil...| 5615|
|Mais de dois milh...| 3688|
|Demais bancos e f...| 3293|
|    Grupo Secundário| 3117|
|Top 10 - Bancos e...| 1633|
|              Top 10| 1475|
+--------------------+-----+



In [20]:
#Numero de irregularidades (total) de cada tipo "Conglomerado" e "Banco/Financeira"
df.groupBy(F.col('tipo')).count().sort("count", ascending=False).show()

22/01/23 20:39:05 WARN org.apache.spark.scheduler.TaskSetManager: Stage 17 contains a task of very large size (9891 KiB). The maximum recommended task size is 1000 KiB.


+----------------+-----+
|            tipo|count|
+----------------+-----+
|    Conglomerado|41582|
|Banco/financeira|10536|
+----------------+-----+



In [21]:
#Exibindo "Instituição Financeira", "Ano", "Período" e a quantidade de cada tipo de irregularidade
#com exibição particionada pelo nome da Instituição Financeira e ordenada por ano

w0 = Window.partitionBy(F.col('instituicao_financeira')).orderBy('ano')
(df.withColumn("row_number", F.row_number().over(w0)).select(F.col("instituicao_financeira"),
                                                              F.col("ano"),
                                                              F.col("periodo"), 
                                                              F.col("qtd_reclamacoes_reguladas_procedentes"),
                                                              F.col("qtd_reclamacoes_reguladas_outras"),
                                                              F.col("qtd_reclamacoes_nao_reguladas"),
                                                              F.col("qtd_total_reclamacoes"))).show()


+----------------------+----+------------+-------------------------------------+--------------------------------+-----------------------------+---------------------+
|instituicao_financeira| ano|     periodo|qtd_reclamacoes_reguladas_procedentes|qtd_reclamacoes_reguladas_outras|qtd_reclamacoes_nao_reguladas|qtd_total_reclamacoes|
+----------------------+----+------------+-------------------------------------+--------------------------------+-----------------------------+---------------------+
|    BANCO DAYCOVAL S A|2015| 1º semestre|                                    0|                               0|                           33|                   33|
|  SAFRA (conglomerado)|2020|2º trimestre|                                   21|                              45|                            0|                   66|
|  SANTANDER (conglo...|2019|2º trimestre|                                   16|                               3|                            0|                   19|
|   

22/01/23 20:39:06 WARN org.apache.spark.scheduler.TaskSetManager: Stage 20 contains a task of very large size (9891 KiB). The maximum recommended task size is 1000 KiB.


In [22]:
#Filtro por Conglomerados, Bancos e/ou Financeiras que começam com a letra B
df.filter("instituicao_financeira like 'B%' ").show()

+----+--------------------+----------------------+--------------------+------------+-----------------------------+--------------------------------+-------------------------------------+---------------------+----------------+
| ano|           categoria|instituicao_financeira|      irregularidade|     periodo|qtd_reclamacoes_nao_reguladas|qtd_reclamacoes_reguladas_outras|qtd_reclamacoes_reguladas_procedentes|qtd_total_reclamacoes|            tipo|
+----+--------------------+----------------------+--------------------+------------+-----------------------------+--------------------------------+-------------------------------------+---------------------+----------------+
|2015|Menos de dois mil...|    BANCO DAYCOVAL S A|Insatisfação com ...| 1º semestre|                           33|                               0|                                    0|                   33|Banco/financeira|
|2019|Mais de quatro mi...|    BMG (conglomerado)|Bloqueio de marge...|4º trimestre|                

22/01/23 20:39:07 WARN org.apache.spark.scheduler.TaskSetManager: Stage 21 contains a task of very large size (9891 KiB). The maximum recommended task size is 1000 KiB.


In [23]:
#Número de irregularidades de cada tipo, ordenadas de maneira decrescente
df.groupBy(F.col("irregularidade")).count().sort("count", ascending=False).show()

22/01/23 20:39:08 WARN org.apache.spark.scheduler.TaskSetManager: Stage 22 contains a task of very large size (9891 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----+
|      irregularidade|count|
+--------------------+-----+
|Renegociação de d...| 1312|
|Insatisfação com ...| 1239|
|Reclamações relac...| 1190|
|Insatisfação com ...| 1117|
|Restrições cadast...| 1110|
|Irregularidades r...| 1101|
|Recusa de produto...| 1031|
|  Cobranças diversas|  982|
|Insatisfação com ...|  793|
|Descumprimento de...|  782|
|Irregularidades r...|  773|
|Renegociação de d...|  770|
|Cancelamento de c...|  756|
|Outros assuntos n...|  714|
|Oferta ou prestaç...|  687|
|Restrição à reali...|  686|
|Irregularidades r...|  685|
|Concessão de créd...|  624|
|Dificuldade na li...|  614|
|               Golpe|  610|
+--------------------+-----+
only showing top 20 rows



In [24]:
#Exibindo reclamações sobre "Renegociação de dívida (exceto cartão de crédito)", 
#referentes ao 2º semestre de 2014.

df.where((F.col("periodo") == "2º semestre") & (F.col("irregularidade") == "Renegociação de dívida (exceto cartão de crédito)")).show()

+----+--------------------+----------------------+--------------------+-----------+-----------------------------+--------------------------------+-------------------------------------+---------------------+----------------+
| ano|           categoria|instituicao_financeira|      irregularidade|    periodo|qtd_reclamacoes_nao_reguladas|qtd_reclamacoes_reguladas_outras|qtd_reclamacoes_reguladas_procedentes|qtd_total_reclamacoes|            tipo|
+----+--------------------+----------------------+--------------------+-----------+-----------------------------+--------------------------------+-------------------------------------+---------------------+----------------+
|2014|Menos de dois mil...|    BANCO DAYCOVAL S A|Renegociação de d...|2º semestre|                           12|                               0|                                    0|                   12|Banco/financeira|
|2014|Menos de dois mil...|  MIDWAY S.A. - CRÉ...|Renegociação de d...|2º semestre|                     

22/01/23 20:39:09 WARN org.apache.spark.scheduler.TaskSetManager: Stage 25 contains a task of very large size (9891 KiB). The maximum recommended task size is 1000 KiB.


# Salvando o arquivo (em formato parquet)

In [25]:
(df.write.format("parquet")
.option("header", "true")
.option("inferschema", "true")
.save('gs://deloitte_g10/saída/Pyspark/BC-Irregularidades/Irregularidades_2014_2021.parquet')
)

22/01/23 20:39:25 WARN org.apache.spark.scheduler.TaskSetManager: Stage 26 contains a task of very large size (9891 KiB). The maximum recommended task size is 1000 KiB.
